In [6]:
''' Defines the Instant class and some useful functions. '''


class Instant:

    def __init__(self, _id, forecasts=[], observations={}):
        
        self._id = _id
        self.casts = forecasts
        self.obs = observations
#         self.count = len(self.casts)`
        self.as_dict = {'_id': self._id,
                        'forecasts': self.casts,
                        'observations': self.obs
                        }
    
    @property
    def count(self):
        ''' Get the count of the elements in self.casts. '''
        
        return len(self.casts)
    
    @property
    def itslegit(self):
        ''' Check the instant's weathers array's count and if it is 40, then the
        document is returned.

        :param instant: the instant docuemnt to be legitimized
        :type instant: dictionary
        '''
        
        self.count()
        if self.count == 40:
            return True
        else:
            return False
    
    def to_dbncol(self, collection='test'):
        ''' Load the data to the database. 

        :param collection: the collection name
        :type collection: string
        '''

        from config import database
        from db_ops import dbncol

        col = dbncol(client, collection, database=database)
        col.update_one({'_id': self._id}, {'$set': self.as_dict}, upsert=True)
        
    def as_delta(self):
        ''' Create an Instant delta object. It finds the delta between the 
        observation and each forecast and returns a list of deltas. '''
        
        from delta import make_delta
        
        # Loop through the forecasts array and append the return of make_delta()
        # to and deltas list before returning that list.
        return [make_delta(cast, self.obs) for cast in self.casts]

    
def cast_count_all(instants):
    ''' get a tally for the forecast counts per document 

    :param instants: docmuments loaded from the db.instants collection ### NOT Instant class objects
    :type instants: list
    '''
    
    n = 0
    collection_cast_counts = {}

    # Go through each doc in the collection and count the number of items in the forecasts array.
    # Add to the tally for that count.
    for doc in instants:
#         n = count(doc)  ### edited because the docs are not Instant objects
        n = len(doc['forecasts'])
        # Move the legit instants to the permenant database
        if n >= 40:
            load_legit(doc)
        if n in collection_cast_counts:
            collection_cast_counts[n] += 1
        else:
            collection_cast_counts[n] = 1
    return collection_cast_counts


def sweep(instants):
    ''' Move any instant that has a ref_time less than the current next
    ref_time and with self.count less than 40. This is getting rid of the
    instnats that are not and will never be legit. 

    :param instants: a list of Instant objects
    '''
    
    import time
    
    import pymongo
    from pymongo.cursor import Cursor
    from Extract.make_instants import find_data
    from config import client, database
    from db_ops import dbncol
    
    col = dbncol(client, 'instant_temp', database=database)
    n = 0
    # Check the instant type- it could be a dict if it came from the database,
    # or it could be a list if it's a bunch of instant objects, or a pymongo
    # cursor over the database.
    if type(instants) == dict:
        print(type(instants))
        for key, doc in instants:
            if key['instant'] < time.time()-453000:  # 453000 seconds is about 5 days
#                 print('instant out of range')
                col.delete_one(doc)
                n += 1
    elif type(instants) == list:
        print(type(instants))
        for doc in instants:
            if doc['instant'] < time.time()-453000:
#                 print('instant out of range')
                col.delete_one(doc)
                n += 1
    elif type(instants) == pymongo.cursor.Cursor:
        print(type(instants), instants)
        for doc in instants:
            if doc['instant'] < time.time()-453000:
#                 print('instant out of range')
                col.delete_one(doc)
                n += 1
    else:
        print(type(instants))
#     print(f'just deleted {n} documents')
    return

def find_legit(instants):
    
    ''' find the 'legit' instants within the list

#     :param instants: all the instants pulled from the database
#     :type instants: list
#     :return: list of instants with a complete forecasts array
#     '''

    i = [item for item in instants if len(item['forecasts']) >= 40]
    return f'legit list is {len(i)} items long'
#     temp = []
#     for item in instants:
#         if item.itslegit() == False:
# #             temp.append(item)
#         if item.itslegit() == True:
#             temp.append(item)
#     return temp#
    ### maybe you should make the instant documents pulled form the database
    ### represented as Instants in memory. Then you could use the Instant
    ### methods you've been writing.


def load_legit(legit_list):
    ''' Load the 'legit' instants to the remote database and delete from temp.
    This process does not delete the 

    :param collection: the collection you want to pull instants from
    :type collection: pymongo.collection.Collection
    '''

    from pymongo.errors import DuplicateKeyError
    from config import client, database
    from db_ops import dbncol
    from db_ops import copy_docs
    
### this should load to legit_inst in owmap for production ###
#         col = dbncol(client, 'legit_inst', 'owmap')
    col = dbncol(client, 'legit_inst', database=database)
    try:
        col.insert_one(legit_list)
        print('finished loading legit instant document!!')
#         col = dbncol(client, 'instant_temp', database=database)
#         col.delete_one(legit_list)
#         print('finished deleting instant from temp database')
    except DuplicateKeyError:
        col = dbncol(client, 'instant_temp', database=database)
        col.delete_one(legit_list)
        print('finished deleting instant from temp database')
        ### saved for later, when doing it on bulk ###
#     col.insert_many(legit_list)
    # Now go to the temp_instants collection and delete the instants just
    # loaded to legit_inst.
#     col.delete_many(legit_list)
    return

def doc_to_inst(doc):
    ''' Take a document from the instants database and make an Instant object
    out of it.
    
    :param doc: a document from the owmap.legit_inst database
    :type doc: dictionary
    '''
    
    _id = f"{doc['instant']}{doc['zipcode']}"
    forecasts = doc['forecasts']
    observations = doc['weather']
    return Instant(_id, forecasts, observations)

In [7]:
# inst_col = col.find({})  #count_documents({})
# print(len(inst_col))
import config
import db_ops

collection = 'instant_temp'
database = 'owmap'
col = db_ops.dbncol(config.client, collection, database=database)
# cast_count_all(col.find({}))
# sweep(col.find({}))

In [12]:
inst_col = col.find({})  #count_documents({})
doc = inst_col[0]
inst = doc_to_inst(doc)
inst.as_delta()
# inst = Instant(f"{inst_col[0]['instant']}{inst_col[0]['zipcode']}", inst_col[0]['forecasts'], inst_col[0]['weather'])
# inst.as_delta()
# find_legit(inst_col)

[{'sunset_time': -1587600068,
  'sunrise_time': -1587551933,
  'clouds': -1,
  'rain': {},
  'snow': {},
  'wind': {'speed': -0.020000000000000018, 'deg': -10},
  'humidity': 29,
  'pressure': {'press': 1},
  'temperature': {'temp': -5.180000000000007,
   'temp_max': -5.839999999999975,
   'temp_min': -4.199999999999989},
  'status': 0,
  'detailed_status': 0,
  'weather_code': 0,
  'weather_icon_name': 0,
  'visibility_distance': 999999,
  'dewpoint': 999999,
  'humidex': 999999,
  'heat_index': 999999,
  'time_to_instant': 614},
 {'sunset_time': -1587600068,
  'sunrise_time': -1587551933,
  'clouds': -1,
  'rain': {},
  'snow': {},
  'wind': {'speed': -0.020000000000000018, 'deg': -10},
  'humidity': 29,
  'pressure': {'press': 1},
  'temperature': {'temp': -5.180000000000007,
   'temp_max': -5.839999999999975,
   'temp_min': -4.199999999999989},
  'status': 0,
  'detailed_status': 0,
  'weather_code': 0,
  'weather_icon_name': 0,
  'visibility_distance': 999999,
  'dewpoint': 999999

In [12]:
def key_list(d=dict):
    ''' Write the dict keys to a list and return that list 
    
    :param d: A python dictionary
    :return: A list fo the keys from the python dictionary
    '''
    
    keys = d.keys()
    key_list = []
    for k in keys:
        key_list.append(k)
    return key_list


def all_keys(d):
    ''' Get all the dicitonary and nested "dot format" nested dictionary keys
    from a dict, add them to a list, return the list.
    
    :param d: A python dictionary
    :return: A list of every key in the dictionary
    '''
    keys = []    
    for key, value in d.items():
        if isinstance(d[key], dict):
            for sub_key in all_keys(value):
                keys.append(f'{key}.{sub_key}')
        else:
            keys.append(str(key))
    return keys

['_id',
 'instant',
 'zipcode',
 'forecasts',
 'weather.sunset_time',
 'weather.sunrise_time',
 'weather.clouds',
 'weather.wind.speed',
 'weather.wind.deg',
 'weather.humidity',
 'weather.pressure.press',
 'weather.pressure.sea_level',
 'weather.temperature.temp',
 'weather.temperature.temp_kf',
 'weather.temperature.temp_max',
 'weather.temperature.temp_min',
 'weather.status',
 'weather.detailed_status',
 'weather.weather_code',
 'weather.weather_icon_name',
 'weather.visibility_distance',
 'weather.dewpoint',
 'weather.humidex',
 'weather.heat_index',
 'weather.time_to_instant']